In [1]:
import pandas as pd
import language_tool_python
from google_play_scraper import app
from bs4 import BeautifulSoup
import re
import numpy as np

In [2]:
def clean_text(text):
    
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text(separator=" ")

    cleaned_text = re.sub(r"Ã¢Å|[^\w\s.,/'’-`]", "", cleaned_text)
    
    cleaned_text = re.sub(r"\s+", " ", cleaned_text).strip()
    
    return cleaned_text

In [3]:
tool = language_tool_python.LanguageTool('en-US')  

In [4]:
xlsx_file = 'grammar.xlsx'  
data = pd.read_excel(xlsx_file)
app_id_column = 'app id'  
description_column = 'description'   
edited_description_column = 'edited description'
mistakes_column = 'mistakes'  
word_count_column = 'word count'

In [5]:
descriptions = []


for index, row in data.iterrows():
    app_id = row[app_id_column]
    try:
        result = app(app_id)
        description = result['description']
        descriptions.append(description)
        print(f'{app_id} is done')
    except Exception as e:
        print(f"App ID '{app_id}' not found. Skipping...")
        descriptions.append(np.nan)

data[description_column] = descriptions
data.to_excel(xlsx_file, index=False)

    


com.eaf.ethereum_angel_faucet is done
com.rayo.savecurrentlocation is done
com.mapswithme.maps.pro is done
dev.egl.com.intensidadbluetooth is done
com.geozilla.family is done
com.gotv.nflgamecenter.us.lite is done
deezer.android.app is done
co.windyapp.android is done
com.gpsmapcamera.geotagginglocationonphoto is done
com.stankomashstroi.industry is done
phonetracker.locationtracker.gpstracker.familylocator is done
com.mytruckingskills.v2 is done
com.infogama.codesverify is done
com.sign.png is done
com.plongo.nvcharts is done
com.toralabs.deviceinfo is done
com.jeyluta.timestampcamerafree is done
com.mapmyride.android2 is done
com.tr.kavuntek.thermometer is done
com.smsactivate is done
com.flitto.app is done
com.who.tracky is done
com.wilysis.cellinfo is done
ru.andr7e.deviceinfohw is done
App ID 'com.eth.server.miner.ethereum.cloud.mining.cryptocoin.app' not found. Skipping...
com.magic.avatar is done
com.google.android.apps.googleassistant is done
com.RAWorks.BM is done
com.mageeks.

In [13]:
data.dropna(subset=[description_column], inplace=True)
edited_descriptions = []
# print(descriptions)


for index, row in data.iterrows():
        description = row[description_column]
        cleaned_description = clean_text(description)
        edited_descriptions.append(cleaned_description)
        

data[edited_description_column] = edited_descriptions
data.to_excel(xlsx_file, index=False)

In [15]:
mistake_counts = []
word_counts= []

for index, row in data.iterrows():
    print(index)
    description = row[edited_description_column]
    matches = tool.check(description)
    
    num_mistakes = len(matches)
    mistake_counts.append(num_mistakes)

    num_words = len(description.split())
    word_counts.append(num_words)

data[mistakes_column] = mistake_counts
data[word_count_column] = word_counts

data.to_excel(xlsx_file, index=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
25
26
27
28
29
30
31
32
33
34
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
